In [2]:
import requests
import time
import sqlite3
import pandas as pd
import smtplib
import matplotlib.pyplot as plt
from datetime import datetime


In [1]:

# Initialize SQLite database connection
conn = sqlite3.connect("weather_data.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute('''CREATE TABLE IF NOT EXISTS weather_data (
                    city TEXT,
                    date DATE,
                    main TEXT,
                    temp REAL,
                    feels_like REAL,
                    max_temp REAL,
                    min_temp REAL,
                    rain_percentage REAL
                )''')

# Configurations
API_KEY = "dbc2dfa8b7ebd6327c4203d3e15f9362"
ALERT_THRESHOLD = 35.0  # Example threshold

# Define a function to retrieve weather forecast data by city name
def get_weather_forecast_by_city(city_name):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city_name}&appid={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to retrieve forecast data for {city_name}. Status Code: {response.status_code}, Response: {response.text}")
        return None

# Analyze the weather forecast for the specified day
def analyze_weather_forecast(forecast_data, user_date):
    weather_summary = {}
    for item in forecast_data['list']:
        timestamp = item['dt']
        main_condition = item['weather'][0]['main']
        temp = item['main']['temp'] - 273.15  # Convert from Kelvin to Celsius
        feels_like = item['main']['feels_like'] - 273.15  # Convert feels like from Kelvin to Celsius
        rain = item.get('rain', {}).get('1h', 0)  # Rain in the last hour
        
        # Collect weather data
        date = time.strftime('%Y-%m-%d', time.gmtime(timestamp))
        if date not in weather_summary:
            weather_summary[date] = {
                "conditions": [],
                "temp": 0,
                "feels_like": 0,
                "rain": 0,
                "count": 0  # Count how many forecasts there are for the day
            }
        
        weather_summary[date]["conditions"].append(main_condition)
        weather_summary[date]["temp"] += temp
        weather_summary[date]["feels_like"] += feels_like
        weather_summary[date]["rain"] += rain
        weather_summary[date]["count"] += 1  # Increase count
            
    # Filter for the specific user-provided date
    return weather_summary.get(user_date, None)

# Continuous loop for user input
while True:
    user_city = input("Enter the city name to check the weather forecast (or type 'exit' to quit): ").strip()
    
    if user_city.lower() == "exit":  # Check for exit condition
        print("Exiting the program.")
        break

    user_date = input("Enter the date in YYYY-MM-DD format (or press Enter for today's date): ").strip()
    if user_date == "":
        user_date = time.strftime('%Y-%m-%d')  # Default to today's date
    
    # Fetch weather forecast data for the specified city
    forecast_data = get_weather_forecast_by_city(user_city)
    if forecast_data:
        weather_summary = analyze_weather_forecast(forecast_data, user_date)
        
        # Display the summary of the day's weather
        print(f"\nWeather Forecast Summary for {user_city} on {user_date}:")
        print("---------------------------------------")
        if weather_summary:
            avg_temp = weather_summary["temp"] / weather_summary["count"]
            avg_feels_like = weather_summary["feels_like"] / weather_summary["count"]
            rain_amount = weather_summary["rain"]  # Total rain for the day
            
            # Determine the most frequent weather condition
            most_common_condition = max(set(weather_summary["conditions"]), key=weather_summary["conditions"].count)

            # Print data in sequence
            print(f"{user_date}:")
            print(f"  Main Condition: {most_common_condition}")
            print(f"  Avg Temp: {avg_temp:.1f}°C")
            print(f"  Avg Feels Like: {avg_feels_like:.1f}°C")
            print(f"  Total Rain: {rain_amount:.1f}mm")
        else:
            print(f"No data available for {user_date}.")
        print("---------------------------------------\n")

# Close the database connection after exiting the loop
conn.close()


Enter the city name to check the weather forecast (or type 'exit' to quit): kolhapur
Enter the date in YYYY-MM-DD format (or press Enter for today's date): 

Weather Forecast Summary for kolhapur on 2024-10-26:
---------------------------------------
2024-10-26:
  Main Condition: Clouds
  Avg Temp: 23.7°C
  Avg Feels Like: 23.8°C
  Total Rain: 0.0mm
---------------------------------------

Enter the city name to check the weather forecast (or type 'exit' to quit): kolhapur
Enter the date in YYYY-MM-DD format (or press Enter for today's date): 2024-10-27

Weather Forecast Summary for kolhapur on 2024-10-27:
---------------------------------------
2024-10-27:
  Main Condition: Clouds
  Avg Temp: 23.8°C
  Avg Feels Like: 24.0°C
  Total Rain: 0.0mm
---------------------------------------

Enter the city name to check the weather forecast (or type 'exit' to quit): exit
Exiting the program.
